In [1]:
import copy
from pathlib import Path

import numpy as np
import pandas as pd

In [2]:
from tiled.client import from_uri
client = from_uri("http://localhost:8000/api")

ConnectError: [WinError 10061] No connection could be made because the target machine actively refused it

In [4]:
from pathlib import Path
DATA_PATH = Path("aimmdb/data/feff/65272_C_007")
print("Data Path:", DATA_PATH)

Data Path: AIMMDB\data\feff\65272_C_007


In [7]:
from load_FEFF_Data import load_feff_data

load_feff_data(DATA_PATH)

FileNotFoundError: [Errno 2] No such file or directory: 'AIMMDB\\data\\feff\\65272_C_007\\xmu.dat'

In [51]:
# data path
import pathlib

feff = load_feff_data(pathlib.Path("D:/BNL/AIMMDB/FEFF_Data/65272_C_007"))
feff


Files opened:  D:\BNL\AIMMDB\FEFF_Data\65272_C_007\feff.inp D:\BNL\AIMMDB\FEFF_Data\65272_C_007\feff.out D:\BNL\AIMMDB\FEFF_Data\65272_C_007\xmu.dat


(      omega       e     k        mu       mu0       chi
 0   274.278 -11.491 -0.80  0.008794  0.011493 -0.002699
 1   274.741 -11.028 -0.72  0.010453  0.014542 -0.004089
 2   275.156 -10.613 -0.64  0.012638  0.018913 -0.006275
 3   275.521 -10.247 -0.56  0.015644  0.025455 -0.009811
 4   275.838  -9.930 -0.48  0.020026  0.035790 -0.015765
 ..      ...     ...   ...       ...       ...       ...
 95  325.002  39.234  3.56  0.981847  1.006860 -0.025010
 96  326.094  40.325  3.60  0.986115  1.001850 -0.015738
 97  327.197  41.428  3.64  0.989862  0.996706 -0.006844
 98  328.313  42.544  3.68  0.992953  0.991381  0.001572
 99  329.440  43.672  3.72  0.995197  0.985933  0.009264
 
 [100 rows x 6 columns],
 {'feff.inp': 'TITLE 65272_C_007\n\nEDGE      K\nS02       1.0\nCOREHOLE  RPA\nCONTROL   1 1 1 1 1 1\n\nXANES     4 0.04 0.1\n\nFMS       9.0\nEXCHANGE  0 0.0 0.0 2\nSCF       7.0 1 100 0.2 3\nRPATH     -1\n\nPOTENTIALS\n*\tipot\tZ\telement\tl_scmt\tl_fms\n0\t6\tC\t-1\t-1\t8\n1\t8\tO\t-1\

In [11]:
def ingest_feff(client, df, verbose=False):
    """
    Upload the FEFF dataset to database
    """

    for (name, prep), g in df.groupby(["sample.name", "sample.prep"]):
        if verbose:
            print(f"{name}: {prep}, {len(g)}")

        sample_id = client.write_sample({"name" : name, "prep" : prep})

        for i, row in g.iterrows():
            feff_df, _ = read_dat(row.file)
            metadata = row.metadata
            metadata["dataset"] = "feff"
            metadata["sample_id"] = sample_id
            client["uid"].write_dataframe(feff_df, metadata=metadata, specs=["FEFF"])

In [12]:
print("starting ingestion...")
ingest_feff(client, feff, verbose=True)
print("finished.")

starting ingestion...


NameError: name 'client' is not defined

In [13]:
FEFFInputMetadata.schema()

{'title': 'FEFFInputMetadata',
 'type': 'object',
 'properties': {'element': {'$ref': '#/definitions/XDIElement'},
  'measurement_type': {'default': 'xas',
   'allOf': [{'$ref': '#/definitions/MeasurementEnum'}]},
  'dataset': {'title': 'Dataset', 'type': 'string'},
  'sample_id': {'title': 'Sample Id', 'type': 'string'},
  'input_script': {'title': 'Input Script', 'type': 'string'}},
 'required': ['element', 'dataset', 'sample_id', 'input_script'],
 'definitions': {'XDIElement': {'title': 'XDIElement',
   'type': 'object',
   'properties': {'symbol': {'title': 'Symbol', 'type': 'string'},
    'edge': {'title': 'Edge', 'type': 'string'}},
   'required': ['symbol', 'edge']},
  'MeasurementEnum': {'title': 'MeasurementEnum',
   'description': 'An enumeration.',
   'enum': ['xas', 'rixs'],
   'type': 'string'}}}

In [14]:
FEFFOutputMetadata.schema()

{'title': 'FEFFOutputMetadata',
 'type': 'object',
 'properties': {'element': {'$ref': '#/definitions/XDIElement'},
  'measurement_type': {'default': 'xas',
   'allOf': [{'$ref': '#/definitions/MeasurementEnum'}]},
  'dataset': {'title': 'Dataset', 'type': 'string'},
  'sample_id': {'title': 'Sample Id', 'type': 'string'},
  'output_script': {'title': 'Output Script', 'type': 'string'}},
 'required': ['element', 'dataset', 'sample_id', 'output_script'],
 'definitions': {'XDIElement': {'title': 'XDIElement',
   'type': 'object',
   'properties': {'symbol': {'title': 'Symbol', 'type': 'string'},
    'edge': {'title': 'Edge', 'type': 'string'}},
   'required': ['symbol', 'edge']},
  'MeasurementEnum': {'title': 'MeasurementEnum',
   'description': 'An enumeration.',
   'enum': ['xas', 'rixs'],
   'type': 'string'}}}

In [15]:
# we will enforce that XAS metadata satisfies the following schema
ExperimentalXASMetadata.schema()

{'title': 'ExperimentalXASMetadata',
 'type': 'object',
 'properties': {'element': {'$ref': '#/definitions/XDIElement'},
  'measurement_type': {'default': 'xas',
   'allOf': [{'$ref': '#/definitions/MeasurementEnum'}]},
  'dataset': {'title': 'Dataset', 'type': 'string'},
  'sample_id': {'title': 'Sample Id', 'type': 'string'},
  'facility': {'$ref': '#/definitions/FacilityMetadata'},
  'beamline': {'$ref': '#/definitions/BeamlineMetadata'}},
 'required': ['element', 'dataset', 'facility', 'beamline'],
 'definitions': {'XDIElement': {'title': 'XDIElement',
   'type': 'object',
   'properties': {'symbol': {'title': 'Symbol', 'type': 'string'},
    'edge': {'title': 'Edge', 'type': 'string'}},
   'required': ['symbol', 'edge']},
  'MeasurementEnum': {'title': 'MeasurementEnum',
   'description': 'An enumeration.',
   'enum': ['xas', 'rixs'],
   'type': 'string'},
  'FacilityMetadata': {'title': 'FacilityMetadata',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': '

In [ ]:
client["uid"]

In [ ]:
# with the correct metadata we can write to the server
# NOTE this doesn't prevent you from writing garbage but does help
df = pd.DataFrame({"a" : np.random.rand(100), "b" : np.random.rand(100)})
metadata = {"dataset" : "feff", "foo" : "bar", "element" : {"symbol" : "Au", "edge" : "K"}, "facility" : {"name" : "ALS"}, "beamline" : {"name" : "8.0.1"}}
node = client["uid"].write_dataframe(df, metadata=metadata, specs=["FEFF"])
node